In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

# Prepare Data

In [23]:
data_dir = "../../InterpolationBaseline/data/Oct0123_Dec3123/"
data_files = [file for file in os.listdir(data_dir) if file.endswith(".csv")]
data_oct = []
data_nov = []
data_dec = []

for file in data_files:
    df = pd.read_csv(data_dir + file)
    df.loc[df["pm25"] < 0, "pm25"] = 0

    # remove outliers
    if df["pm25"].max() > 500:
        print("Outlier dropped")
        continue

    # decompose timestamp
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="mixed")
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["month"] = df["timestamp"].dt.month
    df["year"] = df["timestamp"].dt.year

    df = df.loc[:, ["year", "month", "day", "hour", "pm25", "longitude", "latitude"]]
    df = df.groupby(["year", "month", "day", "hour"]).mean().reset_index(drop=False)

    if len(df) < 24 * (31 + 30 + 31):
        print("Missing Data")
        continue
    else:
        df_oct = df[df["month"] == 10]
        df_nov = df[df["month"] == 11]
        df_dec = df[df["month"] == 12]
        data_oct.append(df_oct.loc[:, ["pm25", "longitude", "latitude"]].to_numpy())
        data_nov.append(df_nov.loc[:, ["pm25", "longitude", "latitude"]].to_numpy())
        data_dec.append(df_dec.loc[:, ["pm25", "longitude", "latitude"]].to_numpy())

data_oct = np.array(data_oct)
data_nov = np.array(data_nov)
data_dec = np.array(data_dec)
print(data_oct.shape, data_nov.shape, data_dec.shape)

Outlier dropped
Missing Data
Outlier dropped
Outlier dropped
Missing Data
Outlier dropped
Outlier dropped
Missing Data
Outlier dropped
Missing Data
Outlier dropped
Missing Data
Outlier dropped
Outlier dropped
Outlier dropped
(36, 744, 3) (36, 720, 3) (36, 744, 3)


In [28]:
data_dir = "../../InterpolationBaseline/data/Jan0124_Jan2924/"
data_files = [file for file in os.listdir(data_dir) if file.endswith(".csv")]
data_jan = []

for file in data_files:
    df = pd.read_csv(data_dir + file)
    df.loc[df["pm25"] < 0, "pm25"] = 0

    # remove outliers
    if df["pm25"].max() > 500:
        print("Outlier dropped")
        continue

    # decompose timestamp
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="mixed")
    df["hour"] = df["timestamp"].dt.hour
    df["day"] = df["timestamp"].dt.day
    df["month"] = df["timestamp"].dt.month
    df["year"] = df["timestamp"].dt.year

    df = df.loc[:, ["year", "month", "day", "hour", "pm25", "longitude", "latitude"]]
    df = df.groupby(["year", "month", "day", "hour"]).mean().reset_index(drop=False)

    if len(df) < 24 * 30:
        print("Missing Data")
        continue
    else:
        data_jan.append(df.loc[:, ["pm25", "longitude", "latitude"]].to_numpy())

data_jan = np.array(data_jan)
print(data_jan.shape)

Outlier dropped
Outlier dropped
Outlier dropped
Outlier dropped
Outlier dropped
Outlier dropped
Missing Data
(45, 722, 3)


In [32]:
class Geo_LSTM_Dataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
            
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.Y[index]
    
def construct_dataloader(data, seed=42, window_size=24, batch_size=256):
    np.random.seed(seed)

    n_stations = data.shape[0]
    permutation = np.random.permutation(n_stations)
    data_train = data[permutation[:int(n_stations * 0.8)]]
    data_test = data[permutation[int(n_stations * 0.8):]]
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    # construct training data
    for label_index in range(len(data_train)):
        X = np.concatenate([data_train[:label_index], data_train[label_index + 1:]], axis=0)
        Y = data_train[label_index: label_index+1]
        RLat = torch.from_numpy(Y[0, 0, 1] - X[:, 0, 1])
        RLon = torch.from_numpy(Y[0, 0, 2] - X[:, 0, 2])
        for t in range(window_size-1, X.shape[1]):
            history_readings = torch.from_numpy(X[:, t-window_size+1:t+1, 0])
            target_reading = Y[0, t, 0]
            X_train.append((history_readings, RLat, RLon))
            Y_train.append(target_reading)
    Y_train = torch.tensor(Y_train)
    train_dataest = Geo_LSTM_Dataset(X_train, Y_train)
    train_loader = DataLoader(train_dataest, batch_size=batch_size, shuffle=True)

    # construct testing data
    for label_index in range(len(data_test)):
        RLat = torch.from_numpy(data_test[label_index, 0, 1] - data_train[:, 0, 1])
        RLon = torch.from_numpy(data_test[label_index, 0, 2] - data_train[:, 0, 2])
        for t in range(window_size-1, data_test.shape[1]):
            history_readings = torch.from_numpy(data_train[:, t-window_size+1:t+1, 0])
            target_reading = data_test[label_index, t, 0]
            X_test.append((history_readings, RLat, RLon))
            Y_test.append(target_reading)
    Y_test = torch.tensor(Y_test)
    test_dataest = Geo_LSTM_Dataset(X_test, Y_test)
    test_loader = DataLoader(test_dataest, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader


# Construct Model

In [33]:
class Geo_Layer(nn.Module):
    def __init__(self, K=4):
        super(Geo_Layer, self).__init__()
        self.K = K

    def forward(self, X):
        # history_readings: (batch_size, n_stations, window_size)
        # RLat: (batch_size, n_stations)
        # RLon: (batch_size, n_stations)
        history_readings, RLat, RLon = X
        batch_size, n_stations, window_size = history_readings.shape

        # RDist, Rank, R_A: (batch_size, n_stations)
        RDist = torch.sqrt(RLat**2 + RLon**2)
        indice = torch.argsort(RDist)[:, :self.K]   # (batch_size, K)
        nearby_readings = history_readings[torch.arange(batch_size)[:, None], indice]

        return nearby_readings
    
class Geo_LSTM(nn.Module):
    def __init__(self, K=4, num_layers=4, hidden_size=128, fc_hidden=1024):
        super(Geo_LSTM, self).__init__()
        self.geo_layer = Geo_Layer(K)
        self.lstm = nn.LSTM(input_size=K, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        self.fc = nn.Sequential(*[
            nn.Linear(hidden_size, fc_hidden),
            nn.ReLU(),
            nn.Linear(fc_hidden, 1)
        ])

    def forward(self, X):
        # nearby_readings: (batch_size, window_size, K)
        nearby_readings = self.geo_layer(X).permute(0, 2, 1).float()
        # output: (batch_size, window_size, hidden_size) -> (batch_size, 1)
        output, _ = self.lstm(nearby_readings)
        output = self.fc(output[:, -1, :]).squeeze()
        return output


# Train

In [34]:
batch_size = 32
epochs = 100
lr = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## October

In [40]:
geo_lstm = Geo_LSTM().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(geo_lstm.parameters(), lr=lr)
train_loader, test_loader = construct_dataloader(data_oct, seed=42, window_size=24, batch_size=batch_size)
best_loss = 1e10
save_path = f"./model_weights/geolstm_oct.pt"
for epoch in range(epochs):
    geo_lstm.train()
    for X, Y in train_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        optimizer.zero_grad()
        Y_pred = geo_lstm(X)
        loss = criterion(Y_pred, Y.float())
        loss.backward()
        optimizer.step()

    # evaluate on train test set
    with torch.no_grad():
        train_loss = 0
        for X, Y in train_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            train_loss += criterion(Y_pred, Y.float()).item()
        train_loss /= len(train_loader)

        test_loss = 0
        for X, Y in test_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            test_loss += criterion(Y_pred, Y.float()).item()
        test_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    # save model
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(geo_lstm.state_dict(), save_path)
        print(f"Model saved at {save_path}")

Epoch 1/100, Train Loss: 12.3159, Test Loss: 18.7283
Model saved at ./model_weights/geolstm_oct.pt
Epoch 2/100, Train Loss: 9.4421, Test Loss: 12.6372
Model saved at ./model_weights/geolstm_oct.pt
Epoch 3/100, Train Loss: 8.4362, Test Loss: 14.8366
Epoch 4/100, Train Loss: 7.6994, Test Loss: 12.4877
Model saved at ./model_weights/geolstm_oct.pt
Epoch 5/100, Train Loss: 8.1927, Test Loss: 11.6048
Model saved at ./model_weights/geolstm_oct.pt
Epoch 6/100, Train Loss: 7.5247, Test Loss: 16.3910
Epoch 7/100, Train Loss: 7.5953, Test Loss: 17.3382
Epoch 8/100, Train Loss: 6.2702, Test Loss: 13.2843
Epoch 9/100, Train Loss: 7.3553, Test Loss: 11.5642
Model saved at ./model_weights/geolstm_oct.pt
Epoch 10/100, Train Loss: 7.1320, Test Loss: 12.8134
Epoch 11/100, Train Loss: 6.5325, Test Loss: 13.4231
Epoch 12/100, Train Loss: 6.3462, Test Loss: 17.3798
Epoch 13/100, Train Loss: 5.8520, Test Loss: 12.4632
Epoch 14/100, Train Loss: 5.9397, Test Loss: 12.8463
Epoch 15/100, Train Loss: 5.2952, Te

In [42]:
Y_pred_all = []
Y_true_all = []
geo_lstm.load_state_dict(torch.load(save_path))
geo_lstm.eval()
with torch.no_grad():
    for X, Y in test_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        Y_pred = geo_lstm(X)
        Y_pred_all.append(Y_pred.cpu().numpy())
        Y_true_all.append(Y.cpu().numpy())
Y_pred_all = np.concatenate(Y_pred_all)
Y_true_all = np.concatenate(Y_true_all)
RMSE = np.sqrt(np.mean((Y_pred_all - Y_true_all)**2))
CVRMSE = RMSE / Y_true_all.mean()
MAE = np.mean(np.abs(Y_pred_all - Y_true_all))
R2 = 1 - np.sum((Y_pred_all - Y_true_all)**2) / np.sum((Y_true_all - Y_true_all.mean())**2)
print("RMSE:", RMSE)
print("CVRMSE:", CVRMSE)
print("MAE:", MAE)
print("R2:", R2)

RMSE: 3.3997461940033276
CVRMSE: 0.3301037150709755
MAE: 2.497493799006621
R2: 0.6297184838299943


## November

In [37]:
geo_lstm = Geo_LSTM().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(geo_lstm.parameters(), lr=lr)
train_loader, test_loader = construct_dataloader(data_nov, seed=42, window_size=24, batch_size=batch_size)
best_loss = 1e10
save_path = f"./model_weights/geolstm_nov.pt"
for epoch in range(epochs):
    geo_lstm.train()
    for X, Y in train_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        optimizer.zero_grad()
        Y_pred = geo_lstm(X)
        loss = criterion(Y_pred, Y.float())
        loss.backward()
        optimizer.step()

    # evaluate on train test set
    with torch.no_grad():
        train_loss = 0
        for X, Y in train_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            train_loss += criterion(Y_pred, Y.float()).item()
        train_loss /= len(train_loader)

        test_loss = 0
        for X, Y in test_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            test_loss += criterion(Y_pred, Y.float()).item()
        test_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    # save model
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(geo_lstm.state_dict(), save_path)
        print(f"Model saved at {save_path}")

Epoch 1/100, Train Loss: 13.1665, Test Loss: 23.3029
Model saved at ./model_weights/geolstm_nov.pt
Epoch 2/100, Train Loss: 12.1570, Test Loss: 18.7262
Model saved at ./model_weights/geolstm_nov.pt
Epoch 3/100, Train Loss: 14.6990, Test Loss: 17.0579
Model saved at ./model_weights/geolstm_nov.pt
Epoch 4/100, Train Loss: 11.9829, Test Loss: 20.0668
Epoch 5/100, Train Loss: 10.0619, Test Loss: 18.1488
Epoch 6/100, Train Loss: 9.5325, Test Loss: 22.5031
Epoch 7/100, Train Loss: 9.5475, Test Loss: 23.7483
Epoch 8/100, Train Loss: 8.7780, Test Loss: 26.1758
Epoch 9/100, Train Loss: 8.7119, Test Loss: 19.4176
Epoch 10/100, Train Loss: 8.2438, Test Loss: 23.8853
Epoch 11/100, Train Loss: 6.4288, Test Loss: 22.0219
Epoch 12/100, Train Loss: 6.6705, Test Loss: 22.5354
Epoch 13/100, Train Loss: 7.6512, Test Loss: 19.5138
Epoch 14/100, Train Loss: 6.0378, Test Loss: 23.5713
Epoch 15/100, Train Loss: 5.7781, Test Loss: 22.5898
Epoch 16/100, Train Loss: 5.4623, Test Loss: 22.8205
Epoch 17/100, Trai

In [38]:
Y_pred_all = []
Y_true_all = []
geo_lstm.load_state_dict(torch.load(save_path))
geo_lstm.eval()
with torch.no_grad():
    for X, Y in test_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        Y_pred = geo_lstm(X)
        Y_pred_all.append(Y_pred.cpu().numpy())
        Y_true_all.append(Y.cpu().numpy())
Y_pred_all = np.concatenate(Y_pred_all)
Y_true_all = np.concatenate(Y_true_all)
RMSE = np.sqrt(np.mean((Y_pred_all - Y_true_all)**2))
CVRMSE = RMSE / Y_true_all.mean()
MAE = np.mean(np.abs(Y_pred_all - Y_true_all))
R2 = 1 - np.sum((Y_pred_all - Y_true_all)**2) / np.sum((Y_true_all - Y_true_all.mean())**2)
print("RMSE:", RMSE)
print("CVRMSE:", CVRMSE)
print("MAE:", MAE)
print("R2:", R2)

RMSE: 4.135922015085295
CVRMSE: 0.3507384626961344
MAE: 3.0058346139989216
R2: 0.6898451418475822


## December

In [46]:
geo_lstm = Geo_LSTM().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(geo_lstm.parameters(), lr=lr)
train_loader, test_loader = construct_dataloader(data_dec, seed=42, window_size=24, batch_size=batch_size)
best_loss = 1e10
save_path = f"./model_weights/geolstm_dec.pt"
for epoch in range(epochs):
    geo_lstm.train()
    for X, Y in train_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        optimizer.zero_grad()
        Y_pred = geo_lstm(X)
        loss = criterion(Y_pred, Y.float())
        loss.backward()
        optimizer.step()

    # evaluate on train test set
    with torch.no_grad():
        train_loss = 0
        for X, Y in train_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            train_loss += criterion(Y_pred, Y.float()).item()
        train_loss /= len(train_loader)

        test_loss = 0
        for X, Y in test_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            test_loss += criterion(Y_pred, Y.float()).item()
        test_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    # save model
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(geo_lstm.state_dict(), save_path)
        print(f"Model saved at {save_path}")

Epoch 1/100, Train Loss: 39.5002, Test Loss: 46.3392
Model saved at ./model_weights/geolstm_dec.pt
Epoch 2/100, Train Loss: 35.0820, Test Loss: 58.7885
Epoch 3/100, Train Loss: 31.3726, Test Loss: 49.2513
Epoch 4/100, Train Loss: 29.2579, Test Loss: 57.3970
Epoch 5/100, Train Loss: 32.3218, Test Loss: 47.2197
Epoch 6/100, Train Loss: 27.1399, Test Loss: 56.7607
Epoch 7/100, Train Loss: 24.5790, Test Loss: 55.6579
Epoch 8/100, Train Loss: 24.6962, Test Loss: 54.7171
Epoch 9/100, Train Loss: 19.8901, Test Loss: 56.2535
Epoch 10/100, Train Loss: 23.2248, Test Loss: 57.5621
Epoch 11/100, Train Loss: 18.8124, Test Loss: 65.2794
Epoch 12/100, Train Loss: 21.0860, Test Loss: 51.6954
Epoch 13/100, Train Loss: 17.1062, Test Loss: 66.2862
Epoch 14/100, Train Loss: 16.5670, Test Loss: 69.6729
Epoch 15/100, Train Loss: 18.9666, Test Loss: 51.6592
Epoch 16/100, Train Loss: 23.7683, Test Loss: 61.9807
Epoch 17/100, Train Loss: 17.5779, Test Loss: 60.4821
Epoch 18/100, Train Loss: 12.3890, Test Loss:

In [47]:
Y_pred_all = []
Y_true_all = []
geo_lstm.load_state_dict(torch.load(save_path))
geo_lstm.eval()
with torch.no_grad():
    for X, Y in test_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        Y_pred = geo_lstm(X)
        Y_pred_all.append(Y_pred.cpu().numpy())
        Y_true_all.append(Y.cpu().numpy())
Y_pred_all = np.concatenate(Y_pred_all)
Y_true_all = np.concatenate(Y_true_all)
RMSE = np.sqrt(np.mean((Y_pred_all - Y_true_all)**2))
CVRMSE = RMSE / Y_true_all.mean()
MAE = np.mean(np.abs(Y_pred_all - Y_true_all))
R2 = 1 - np.sum((Y_pred_all - Y_true_all)**2) / np.sum((Y_true_all - Y_true_all.mean())**2)
print("RMSE:", RMSE)
print("CVRMSE:", CVRMSE)
print("MAE:", MAE)
print("R2:", R2)

RMSE: 6.82116797296474
CVRMSE: 0.4050034760698666
MAE: 4.724538655543015
R2: 0.6606058691141251


## January

In [48]:
geo_lstm = Geo_LSTM().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(geo_lstm.parameters(), lr=lr)
train_loader, test_loader = construct_dataloader(data_jan, seed=42, window_size=24, batch_size=batch_size)
best_loss = 1e10
save_path = f"./model_weights/geolstm_jan.pt"
for epoch in range(epochs):
    geo_lstm.train()
    for X, Y in train_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        optimizer.zero_grad()
        Y_pred = geo_lstm(X)
        loss = criterion(Y_pred, Y.float())
        loss.backward()
        optimizer.step()

    # evaluate on train test set
    with torch.no_grad():
        train_loss = 0
        for X, Y in train_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            train_loss += criterion(Y_pred, Y.float()).item()
        train_loss /= len(train_loader)

        test_loss = 0
        for X, Y in test_loader:
            X = (X[0].to(device), X[1].to(device), X[2].to(device))
            Y = Y.to(device)
            Y_pred = geo_lstm(X)
            test_loss += criterion(Y_pred, Y.float()).item()
        test_loss /= len(test_loader)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

    # save model
    if test_loss < best_loss:
        best_loss = test_loss
        torch.save(geo_lstm.state_dict(), save_path)
        print(f"Model saved at {save_path}")

Epoch 1/100, Train Loss: 13.1094, Test Loss: 24.2545
Model saved at ./model_weights/geolstm_jan.pt
Epoch 2/100, Train Loss: 12.4818, Test Loss: 27.9262
Epoch 3/100, Train Loss: 11.4570, Test Loss: 24.2141
Model saved at ./model_weights/geolstm_jan.pt
Epoch 4/100, Train Loss: 12.0604, Test Loss: 24.5899
Epoch 5/100, Train Loss: 10.1727, Test Loss: 25.3175
Epoch 6/100, Train Loss: 10.7114, Test Loss: 25.4317
Epoch 7/100, Train Loss: 10.0561, Test Loss: 27.6288
Epoch 8/100, Train Loss: 8.5764, Test Loss: 27.1946
Epoch 9/100, Train Loss: 7.7720, Test Loss: 27.9569
Epoch 10/100, Train Loss: 8.2450, Test Loss: 24.9531
Epoch 11/100, Train Loss: 8.3144, Test Loss: 30.7069
Epoch 12/100, Train Loss: 8.3164, Test Loss: 25.1912
Epoch 13/100, Train Loss: 6.7367, Test Loss: 29.6212
Epoch 14/100, Train Loss: 6.2207, Test Loss: 27.7666
Epoch 15/100, Train Loss: 6.7318, Test Loss: 27.3778
Epoch 16/100, Train Loss: 5.2372, Test Loss: 28.4683
Epoch 17/100, Train Loss: 6.0171, Test Loss: 27.9554
Epoch 18/

In [49]:
Y_pred_all = []
Y_true_all = []
geo_lstm.load_state_dict(torch.load(save_path))
geo_lstm.eval()
with torch.no_grad():
    for X, Y in test_loader:
        X = (X[0].to(device), X[1].to(device), X[2].to(device))
        Y = Y.to(device)
        Y_pred = geo_lstm(X)
        Y_pred_all.append(Y_pred.cpu().numpy())
        Y_true_all.append(Y.cpu().numpy())
Y_pred_all = np.concatenate(Y_pred_all)
Y_true_all = np.concatenate(Y_true_all)
RMSE = np.sqrt(np.mean((Y_pred_all - Y_true_all)**2))
CVRMSE = RMSE / Y_true_all.mean()
MAE = np.mean(np.abs(Y_pred_all - Y_true_all))
R2 = 1 - np.sum((Y_pred_all - Y_true_all)**2) / np.sum((Y_true_all - Y_true_all.mean())**2)
print("RMSE:", RMSE)
print("CVRMSE:", CVRMSE)
print("MAE:", MAE)
print("R2:", R2)

RMSE: 4.925393149814979
CVRMSE: 0.4488830993025483
MAE: 3.514295097989502
R2: 0.7054906185234536
